In [ ]:
**DETR(DEtection TRansformer)** 기반 객체 탐지 실습을 목표로 합니다.  
`Backbone(CNN) → Transformer(Encoder/Decoder) → Object Query 기반 예측` 흐름을 따라가며,
**출력(`pred_logits`, `pred_boxes`)이 어떤 의미인지**와 **attention이 어디를 보고 있는지**를 직접 확인합니다.

## 학습 목표
- DETR의 **Backbone / Transformer(Encoder·Decoder) / Object Query** 흐름을 코드에서 찾을 수 있다.
- 모델 출력인 `pred_logits`, `pred_boxes`의 **shape와 의미**를 설명할 수 있다.  
  - `pred_logits`: (B, num_queries, num_classes + 1)에서 **+1이 no-object**임을 이해한다.
  - `pred_boxes`: (B, num_queries, 4)가 **정규화 박스(`cx, cy, w, h`)**임을 이해한다.
- 간단한 COCO 샘플 이미지로 **추론 → 신뢰도 필터링 → 박스 복원 → 시각화**까지 한 번에 실행할 수 있다.
- Forward hook으로 캡처한 **Encoder self-attention / Decoder cross-attention**을 시각화하고 해석할 수 있다.

In [ ]:
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):  # (cx,cy,w,h) → (xmin,ymin,xmax,ymax) 변환
    x_c, y_c, w, h = x.unbind(1)  # 각 성분을 분리: x.shape=(N,4) 가정
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),  # 중심좌표에서 좌상/우하 꼭짓점 계산
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)  # (N,4)로 다시 스택

def rescale_bboxes(out_bbox, size):  # 정규화 박스를 픽셀 좌표로 변환
    img_w, img_h = size  # PIL size = (W,H)
    b = box_cxcywh_to_xyxy(out_bbox)  # 우선 xyxy로 변환
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)  # (W,H,W,H) 스케일로 곱해 픽셀 단위로 확장
    return b

In [ ]:
# [DETR] 한 장 이미지 추론 + 신뢰도 필터링 + 박스 스케일 복원
# - 논문처럼 모든 쿼리가 박스를 내지만, 여기서는 시각화를 위해 높은 confidence만 남깁니다.

# propagate through the model
outputs = model(img)  # DETR forward → 로짓/박스 예측

# print keys of outputs
for key in outputs.keys():
    # value를 가져와서 shape 출력
    shape = outputs[key].shape
    print(f"Key : {key}")
    print(f"Shape   : {shape}")
    print("-" * 30)  # 구분선 추가
    # - outputs['pred_logits']: (1, num_queries, num_classes+1) 로짓. 마지막 '+1'은 'no-object' 클래스.
    # - outputs['pred_boxes']: (1, num_queries, 4) 박스 (cx,cy,w,h) 정규화 좌표.
    
# keep only predictions with 0.9+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]  
# 쿼리별 클래스 확률 계산(no-object 제외): shape=(num_queries,num_classes)
# (-1) : 마지막 차원(num_classes+1)에 대해 softmax를 적용합니다.
# [0, :, :-1]  : 각 query에 대해 실제 객체 클래스들에 대한 확률만 남깁니다.
keep = probas.max(-1).values > 0.9  # 각 쿼리의 max class score가 임계값을 넘는 것만 선택


In [ ]:
# [DETR] 내부 텐서(특징맵/어텐션) 추출을 위한 forward hook
# - 논문 분석 포인트:
#   1) backbone CNN 특징맵(conv_features)
#   2) 마지막 encoder layer의 self-attention(enc_attn_weights)
#   3) 마지막 decoder layer의 cross-attention(dec_attn_weights)
# - PyTorch hook을 등록해 forward 중간 결과를 캡처합니다.

conv_features, enc_attn_weights, dec_attn_weights = [], [], []  # hook 결과를 담을 리스트(클로저 변수) 초기화

hooks = [  # 각 모듈에 forward hook 등록
    model.backbone[-2].register_forward_hook(  # CNN backbone의 특정 stage 출력(feature map) 저장
        lambda self, input, output: conv_features.append(output) # lambda 인자들 : 실행문
    ),
    model.transformer.encoder.layers[-1].self_attn.register_forward_hook(  # encoder self-attention의 attention weights(output[1]) 저장
        lambda self, input, output: enc_attn_weights.append(output[1])   # self-attention weight -> output[1]
    ),
    model.transformer.decoder.layers[-1].multihead_attn.register_forward_hook(  # decoder cross-attention의 attention weights(output[1]) 저장
        lambda self, input, output: dec_attn_weights.append(output[1])   # cross-attention weight -> output[1]
    ),
]

# propagate through the model
outputs = model(img)  # hook이 등록된 상태로 forward 실행

for hook in hooks:  # 등록했던 hook 해제(메모리/부작용 방지)
    hook.remove()

In [ ]:
""" Skip한 것들 
    ## 10) Decoder cross-attention 시각화
    ## 11) Encoder self-attention과 feature map 확인
    ## 12) Encoder self-attention을 2D 공간으로 복원
    ## 13) Encoder self-attention 시각화
    ## 14) AttentionVisualizer (Encoder Self-attention) 클래스
    ## 15) Encoder Self-attention 인터랙티브 시각화 실행
"""


